# ETL Pipeline

## Extract Data

In [1]:
# %pip install -q tweepy
# %pip install -q python-decouple

In [2]:
# import necessay packages
import pandas as pd
import tweepy as tw
# from decouple import config

In [3]:
# config
consumer_key = 'ALS2SK37YuglyLYSyGGpfrbmY'
consumer_secret = '2Vyr3pBaHDsNaVX9IslmRHq25xR4zJjQjHpjUFTZzXj74rkvwS'
access_token = '1521183878576230400-4DzxTK7h5tI1gQ1f4QwoEWqfn0FNuV'
access_token_secret = 'sft4cbN41jMtVneJLD794rYRGJraHbb9ug4YELbZHFSyV'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA6ScAEAAAAAnv1iKJAyg2INl1nA7%2BB0z%2FJbpK4%3DbrVth6B429DvCW0vsS4ufO6pLzmsAnhyYpaPRfWGoMWBKc6uRM'
user_id = '90676816'

In [4]:
# connect API
permission = tw.OAuthHandler(consumer_key, consumer_secret) # this object of ours will load the authorization request
permission.set_access_token(access_token, access_token_secret) # get ownership of our access object
client = tw.Client(bearer_token=bearer_token)
api = tw.API(permission) # use the API class, native to the library to generate the connection

In [18]:
query = 'Atletico-MG -filter:retweets OR Atlético Mineiro -filter:retweets OR #Galo -filter:retweets OR #ClubeAtléticoMineiro -filter:retweets'
galo_tweets = api.search_tweets(q=query, tweet_mode='extended', count=100)

id = []
created_at = []
full_text = []
favorite_count = []
retweet_count = []

for twitters in galo_tweets:
    id.append(twitters.id)
    created_at.append(twitters.created_at)
    full_text.append(twitters.full_text)
    favorite_count.append(twitters.favorite_count)
    retweet_count.append(twitters.retweet_count)

    # create dataframe
    df_final = {
      'tweet_id': id, 'created_at':created_at, 'text':full_text, 'likes':favorite_count, 
      'retweets':retweet_count
    }

In [19]:
search_tweets_df = pd.DataFrame(df_final)
search_tweets_df

,tweet_id,entities,created_at,text,likes,retweets
0,1616837664778645504,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-21 16:38:33+00:00,Atlético-MG x Caldense: Galo inicia a busca pe...,8,0
1,1616835539675619331,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-21 16:30:06+00:00,COMEÇANDO DAQUI A 3 HORAS!\n\n16:30\nCampeonat...,0,0
2,1616831566621048832,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-21 16:14:19+00:00,Prevejo vitória com muitos gols para o Atlétic...,34,2
3,1616814792047529988,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-21 15:07:40+00:00,Hoje tem vitória do GALO pelo campeonato Mine...,23,2
4,1616773085016776704,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-21 12:21:56+00:00,Atlético-MG x Caldense AO VIVO: onde assistir ...,0,0
...,...,...,...,...,...,...
95,1613661502048960513,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-12 22:17:37+00:00,Atlético-MG parabeniza aniversariante Nacho Fe...,6,1
96,1613643242062630913,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-12 21:05:03+00:00,Atlético-MG é um dos quatro clubes brasileiros...,5,1
97,1613639016360087552,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-12 20:48:16+00:00,Galo vai leiloar 5 camisas da estreia do campe...,28,0
98,1613596179010289665,"{'hashtags': [], 'symbols': [], 'user_mentions...",2023-01-12 17:58:02+00:00,Corinthians sondou o lateral-esquerdo Guilherm...,366,20


In [17]:
search_tweets_df.coordinates.unique()

array([None], dtype=object)

In [7]:
# empty lists are created that will store the data extracted from the API call
id = []
date = []
text = []
retweets = []
answers = []
likes = []
mentions = []

In [8]:
# API call script with loop and pagination
for tweet in tw.Paginator(
    client.get_users_tweets, id=user_id, exclude=['retweets', 'replies'],
    tweet_fields=['id', 'text', 'created_at', 'public_metrics'], max_results=100).flatten(limit=800):

    # inserting the results in the lists
    id.append(tweet['id'])
    date.append(tweet['created_at'])
    text.append(tweet['text'])
    retweets.append(tweet['public_metrics']['retweet_count'])
    answers.append(tweet['public_metrics']['reply_count'])
    likes.append(tweet['public_metrics']['like_count'])
    mentions.append(tweet['public_metrics']['quote_count'])

    # create dataframe
    results = {
      'tweet_id':id, 'date':date, 'text':text, 'likes':likes, 
      'retweets':retweets, 'answers':answers, 'mentions':mentions
    }

In [9]:
paginator_df = pd.DataFrame(results)
paginator_df

,tweet_id,date,text,likes,retweets,answers,mentions
0,1616836896893669380,2023-01-21 16:35:30+00:00,Ô se voltou! E a gente tá como? Molhando a gar...,134,10,5,0
1,1616834356814684163,2023-01-21 16:25:24+00:00,🥳 Parabéns pra vocês! 🎉\n\n🍰 E teve bolo para ...,504,41,6,2
2,1616829221665472512,2023-01-21 16:05:00+00:00,✊🏾 O @Reinaldoreigalo passou pra lembrar que d...,433,38,18,1
3,1616791602499911680,2023-01-21 13:35:31+00:00,🐔 Três anos do Pitbull com a camisa do #Galo!\...,2239,169,84,64
4,1616783927091380226,2023-01-21 13:05:01+00:00,"Hoje não posso, tem jogo do #Galo! 🖤🤍 https://...",2312,431,54,47
...,...,...,...,...,...,...,...
795,1595794378349346816,2022-11-24 15:00:02+00:00,Daqui a pouco tem estreia do braza e o @goleir...,598,41,11,1
796,1595788076466388992,2022-11-24 14:35:00+00:00,"Em clima de Copa e ritmo de jogo, fala aí pra ...",891,40,52,5
797,1595781311326666752,2022-11-24 14:08:07+00:00,🐔🇧🇷 Rodrigo Lasmar comemorou o pentacampeonato...,537,39,3,1
798,1595772976493785088,2022-11-24 13:35:00+00:00,"#TBTDAMASSA\n\n🐓Fala, torcedor! Lembra da ação...",326,17,1,1


In [10]:
# for twitters in coke_tweets:
#     print(twitters.text)